In [1]:
from keras.models import load_model
from keras_preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix
import cv2 as cv

Using TensorFlow backend.


In [2]:
#加载模型
model1 = load_model("weights/vgg16_0032.h5")
model2 = load_model("weights/vgg16_0029.h5")
model3 = load_model("weights/vgg16_0030.h5")

print("模型加载成功")

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


模型加载成功


In [9]:
#导入数据
def load_data(shape):
    test_path = '测试/t'
    test_batches = ImageDataGenerator(rescale=1/255).flow_from_directory(test_path,
                                      target_size=shape,
                                    classes=["C2F","X2F"],
                                    class_mode="binary",batch_size=10,shuffle=False)
    return test_batches

# 预测
def pred(model,steps,shape):
    steps = steps
    test_batches=load_data(shape)
    pred = model.predict_generator(test_batches, steps=steps, verbose=1)
    pred = pred.ravel()
    pred = list(pred)
    for i in range(len(pred)):
        if pred[i] < 0.5:
            pred[i] = 0
        else:
            pred[i] = 1
    return pred
#投票选出最多的
def vote(lt):
	index1 = 0
	max = 0
	for i in range(len(lt)):
		flag = 0
		for j in range(i+1,len(lt)):
			if lt[j] == lt[i]:
				flag += 1
		if flag > max:
			max = flag
			index1 = i
	return index1
def Ensemble():
    ans = []
    pred1=list(pred(model1,1,(224,224)))
    pred2=list(pred(model2,1,(224,224)))
    pred3=list(pred(model3,1,(224,224)))
    for i in range(len(pred1)):
        ls = []
        ls.append(pred1[i])
        ls.append(pred2[i])
        ls.append(pred3[i])
        ans.append(ls[vote(ls)])
    return ans

#投票得出最终结果
predicts=Ensemble()
# for i in enumerate(predicts):
#     print(i)



test_batches = load_data((224,224))
test_class = np.array([])

files=[]
for i in range(1):
    test_imgs, test_lables = next(test_batches)
    test_class = np.hstack((test_class, test_lables))
    files.append(test_imgs)
print("真实类别：", test_class)
print("预测类别：", predicts)


# 打印混淆矩阵
cm = confusion_matrix(test_class, predicts)

print(cm)

tmp = 0
for i in range(len(cm[0, :])):
    tmp += cm[i][i]
accuracy = tmp / np.sum(cm)
print("acc:", accuracy)


i=0
for images in files:
    for label, pred_label, image in zip(test_class, predicts, images):
        i += 1
        cv.imshow('{} - {} {} {}'.format(
            i, label,
            '==' if label == pred_label
            else '!=', pred_label), image)
cv.waitKey(0)
cv.destroyAllWindows()

Found 10 images belonging to 2 classes.


1/1 [==============================] - 0s 130ms/step


Found 10 images belonging to 2 classes.


1/1 [==============================] - 0s 140ms/step


Found 10 images belonging to 2 classes.


1/1 [==============================] - 0s 130ms/step


Found 10 images belonging to 2 classes.
真实类别： [0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
预测类别： [0, 0, 0, 0, 0, 1, 0, 1, 0, 1]
[[5 0]
 [2 3]]
acc: 0.8
